<a href="https://colab.research.google.com/github/dujest/yolov5-workshop/blob/main/YOLOv5_Hyperparameter_Evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/

In [ ]:
!mkdir ./colab

In [2]:
%cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


# Install Requirements

In [3]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/content/drive/MyDrive/colab/yolov5
     |████████████████████████████████| 1.6 MB 51.5 MB/s 
     |████████████████████████████████| 145 kB 40.4 MB/s 
     |████████████████████████████████| 178 kB 54.5 MB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 138 kB 60.0 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


# Get a Dataset

In [ ]:
!mkdir ../datasets

In [4]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/drive/MyDrive/colab/datasets"

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="hqecDDiHcpNES0AxG9i3")
project = rf.workspace("caitlin-hall").project("shipclassification")
dataset = project.version(3).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/drive/MyDrive/colab/datasets/ShipClassification-3 in yolov5pytorch:: 100%|██████████| 2714/2714 [00:20<00:00, 134.22it/s]


# Hyperparameter Evolution

Hyperparameters in ML control various aspects of training, and finding optimal values for them can be a challenge. Traditional methods like grid searches can quickly become intractable due to
<br>
1) the high dimensional search space,
<br>
2) unknown correlations among the dimensions,
<br>
3) expensive nature of evaluating the fitness at each point.
<br>
These reasons makes GA a suitable candidate for hyperparameter searches.

In [ ]:
!python train.py --epochs 10 --data '/content/drive/MyDrive/colab/datasets/ShipClassification-3/data.yaml' --weights yolov5m.pt --cache --evolve

# Train the data

In [ ]:
!python train.py --img 416 --batch 32 --epochs 150 --data '/content/drive/MyDrive/colab/datasets/ShipClassification-3/data.yaml' --weights yolov5m.pt --cache --hyp "/content/drive/MyDrive/colab/yolov5/runs/evolve/exp/hyp_evolve.yaml"